In [1]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
from tqdm.notebook import tqdm 
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [5]:
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [3]:
driver.maximize_window() # 윈도우 창 최대화

In [9]:
# 스타벅스) 매장 찾기-지역 검색 링크 주소
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [10]:
# 스타벅스) '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click() # 5초 텀을 두고 클릭한다?


In [11]:
# 스타벅스) '서울'-'전체' 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all))).click()

In [41]:
# 스타벅스 html 코드를 파싱하여 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지의 소스 코드 반환
filename = 'starbucks.html'
htmlfile = open(filename, 'w', encoding='UTF-8')
print(html, file=htmlfile)
htmlfile.close()
print(filename + ' 파일 생성됨')

starbucks.html 파일 생성됨


In [42]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# 파싱된 결과를 Beautiful Soup 객체로 생성합니다.
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [43]:
# 매장 정보들을 담고 있는 컨테이너 박스를 찾습니다.
# 컨테이너는 div 태그이고, id가 mCSB_3_container입니다.
container = soup.find('div', id='mCSB_3_container')
storeAll = container.find_all('li')
print('storeAll : %d' % len(storeAll))

storeAll : 615


In [58]:
starbucksData = []

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('strong').text.strip() # 공백 없애주기
    address = store.find('p').text.strip().replace('1522-3232', '')
    imsi = address.split(' ') # split()의 반환타입은 list이다.
    gu = imsi[1]
    latitude = store['data-lat'] # 위도 / 속성은 []로 가져온다.
    longitude = store['data-long'] # 경도
    # print('[' + name + ']')
    # print('[' + address + ']')
    # print(imsi)
    # print(gu)
    # print(latitude)
    # print(longitude)

    starbucksData.append([brand, name, address, gu, latitude, longitude])
    # break
    
print(len(starbucksData))

615


In [80]:
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = ['브랜드', '상호', '주소', '군구', '위도', '경도']
sbDataFrame.head(n=7) #default는 5개 / 반대 메소드 .tail

,브랜드,상호,주소,군구,위도,경도
0,스타벅스,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),강남구,37.501087,127.043069
1,스타벅스,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),강남구,37.510178,127.022223
2,스타벅스,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),강남구,37.5139309,127.0206057
3,스타벅스,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),강남구,37.499517,127.031495
4,스타벅스,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),강남구,37.494668,127.062583
5,스타벅스,봉은사역,서울특별시 강남구 봉은사로 619 (삼성동),강남구,37.515000,127.063196
6,스타벅스,압구정윤성빌딩,서울특별시 강남구 논현로 834 (신사동),강남구,37.5227934,127.0286009


In [84]:
# print(sbDataFrame.isnull().sum())
print('위도 누락 데이터 개수 : %d' % sbDataFrame['위도'].isnull().sum())

위도 누락 데이터 개수 : 0


In [86]:
guList = list(sbDataFrame['군구'].unique())
print('서울시 구 목록 개수 : %d' % len(guList))
print(guList)

서울시 구 목록 개수 : 25
['강남구', '강북구', '강서구', '관악구', '광진구', '금천구', '노원구', '도봉구', '동작구', '마포구', '서대문구', '서초구', '성북구', '송파구', '양천구', '영등포구', '은평구', '종로구', '중구', '강동구', '구로구', '동대문구', '성동구', '용산구', '중랑구']


In [92]:
# 이디야 매장
ediya_url = 'https://ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [93]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 클릭

ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click() # 5초 텀을 두고 클릭한다?


In [ ]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기

In [94]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = '91e7f42cf0a9ac2e61446d7b262a86db'
header = {'Authorization': 'KakaoAK ' + api_key}

In [136]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기
def getGeoCoder(address):
    result = ""
    url = url_header + address
    response = requests.get(url, headers=header)
    # print(response) # 성공시 <Response [200]>으로 리턴 됨
    # print(response.json())
    if response.status_code == 200:
        try:
            result_address = response.json()["documents"][0]["address"]
            result = result_address["y"], result_address["x"]
        except Exception as err:
            return None
    else:
        result = "ERROR[" + str(response.status_code) + "]"

    return result # tuple 타입
# end def

In [135]:
# 한 개의 매장에 대한 테스트를 진행합니다.
#geoInfo = getGeoCoder('서울 중랑구 망우로 460 (망우동)') # 잘 동작하는 주소
geoInfo = getGeoCoder('서울 노원구 한글비석로 409 (상계동) 1~2층') # NoneType이 리턴되는 주소
geoInfo

{'documents': [], 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}


In [1]:
ediyaData = []

for gu in tqdm(guList):
    ediya_search_keyword_css = '#keyword'
    
    # 이디야 주소 검색어 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_keyword_css))).clear()
    
    # 이디야 주소 검색어 입력
    # f"서울 {gu}" : f-string 기법, 서울 중구, 서울 마포구 등등
    WebDriverWait(driver, 10).until(EC.presence_of_element_located
                                    ((By.CSS_SELECTOR, ediya_search_keyword_css))).send_keys(f"서울 {gu}")

    # 이디야 주소 검색 버튼 클릭
    ediya_search_button_css = '#keyword_div > form > button'
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button_css))).click()

    html = driver.page_source # 각 구별 매장 정보를 담고 있는 html 소스 

    # 개별 구에 대한 결과를 파일로 저장해 봅니다.
    # filename = open(f'서울 {gu}.html', 'w', encoding='UTF-8')
    # print(html, file=filename)
    # filename.close()
    # print(f'서울 {gu}.html 파일 생성')

    soup = BeautifulSoup(html, 'html.parser')
    ul_tag = soup.find('ul', id='placesList')

    oneGuEdiyaList = ul_tag.find_all('li')
    for store in oneGuEdiyaList:
        # print(store)
        brand = '이디야'
        name = store.find('dt').text.strip()
        adress = store.find('dd').text.strip()
        imsi = address.split(' ')
        sido = imsi[0]
        gu = imsi[1]

        # 위도/경도 정보가 들어 있는 문자열
        geoInfoString = store.find('a')['onclick']
        geoInfoImsi = geoInfoString # 중간에 변형이 이루어 지므로 임시 복사

        # 넘파이의 nan으로 무의미한 데이터 만들기
        latitude = np.nan # 위도
        longitude = np.nan # 경도
        latLong = ['0', '0']

        if geoInfoString.startswith("panLatTo"): 
            # 올바른 위도/경도가 아님
            try:
                if geoInfoString.index(r"panLatTo('0','0'") == 0: 
                    geoInfo = getGeoCoder(address) 
                    if geoInfo != None:
                        latitude = geoInfo[0] # 위도
                        longitude = geoInfo[1] # 경도
                    else:
                        print(address)                
            except ValueError as err:
                # 올바른 위도/경도입니다.
                latLong = geoInfoString.replace(r"panLatTo('", '').replace(r"');fnMove();", '')
                latLong = latLong.split("','")
        
                latitude = latLong[1] 
                longitude = latLong[0]                
            # end try

        else: # 'panAddTo'으로 시작하는 경우
            geoInfo = getGeoCoder(address) 
            if geoInfo != None:
                latitude = geoInfo[0] # 위도
                longitude = geoInfo[1] # 경도
            else:
                print(address)
        # end if

        # 올바른 위도 경도 형식이 아니면
        if latLong[1] == '0' or latLong[0] == '0':            
            # 카카오 지도 api 이용하여 위도와 경도를 취득합니다.
            geoInfo = getGeoCoder(address) 
            if geoInfo != None:
                latitude = geoInfo[0] # 위도
                longitude = geoInfo[1] # 경도
            else:
                print(address)
        # end if
        
        ediyaData.append([brand, name, address, gu, latitude, longitude, geoInfoImsi]) 
    
        # break
    # print('-'*30)

# end for

print('이디야 매장 개수 : %d' % len(ediyaData))

NameError: name 'tqdm' is not defined

In [129]:
ediyaDataFrame = pd.DataFrame(ediyaData)
ediyaDataFrame.columns = ['브랜드', '상호', '주소', '군구', '위도', '경도', 'geoInfoImsi']
# print(ediyaDataFrame.head())
print(ediyaDataFrame.describe())

# ediyaDataFrame.isnull().sum()

print('위도 누락 데이터 개수 : %d' % ediyaDataFrame['위도'].isnull().sum())
print('경도 누락 데이터 개수 : %d' % ediyaDataFrame['경도'].isnull().sum())


        브랜드    상호                    주소   군구                위도  \
count   573   573                   573  573               573   
unique    1   573                     1    1               271   
top     이디야  원효로점  서울특별시 중랑구 봉화산로 35 1층  중랑구  37.6018095864644   
freq    573     1                   573  573               303   

                      경도                      geoInfoImsi  
count                573                              573  
unique               271                              475  
top     127.078324330908  panLatTo('0','0','5');fnMove();  
freq                 303                                9  
위도 누락 데이터 개수 : 0
경도 누락 데이터 개수 : 0


In [138]:
print('스타 벅스 매장 개수 : %d' % len(sbDataFrame))
print('이디야 매장 개수 : %d' % len(ediyaDataFrame))

스타 벅스 매장 개수 : 615
이디야 매장 개수 : 573


In [139]:
# 임시 컬럼을 제거하고, 두 매장의 데이터 프레임을 합칩니다.
ediyaDataFrame = ediyaDataFrame.drop('geoInfoImsi', axis=1) # axis=1 이면 컬럼
coffeeFrame = pd.concat([sbDataFrame, ediyaDataFrame])
print('전체 매장 개수 : %d' % len(coffeeFrame))

전체 매장 개수 : 1188


In [150]:
filename = 'coffeeList.csv'
# coffeeFrame.to_csv(filename, encoding='CP949', index=False) # 엑셀에서 한글 안깨지고 보려면
coffeeFrame.to_csv(filename, encoding='UTF-8', index=False) # 일반적인 방법
print(filename + '파일 저장됨')

coffeeList.csv파일 저장됨


In [151]:
print(coffeeFrame.duplicated(subset=['위도', '경도']).sum())

302
